This notebook generates a nice anatomy figure of the gad1b line on top of huc:h2b-mCherry

In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import nrrd
import numpy as np
import tifffile as tiff
from PIL import Image, ImageDraw
from scipy.stats import zscore
from skimage import exposure
from scipy.ndimage import zoom

import lotr.plotting as pltltr

COLS = pltltr.COLS
COLS["ring"]

from lotr.em.transformations import ipn_to_mpin
from lotr.data_preprocessing.anatomy import transform_points
from lotr.plotting.anatomy import AtlasPlotter
from bg_atlasapi import BrainGlobeAtlas
from matplotlib.colors import LinearSegmentedColormap

cmap_ring = LinearSegmentedColormap.from_list(
    "ring", [(1.0, 1.0, 1.0, 0.0), (*COLS["ring"], 1.0,)]
)


In [ ]:
# Loading confocal stacks of onr of the fish
# master = Path(r"\\Funes\Shared\experiments\E0038_confocal\v004_ipn_reference\211108_h2b-mcherry_gad1b-gfp\f2")
master = Path(
    r"D:\Shared\experiments\E0038_confocal\v004_ipn_reference\211108_h2b-mcherry_gad1b-gfp\f2"
)
anatomy_h2b, _ = nrrd.read(list(master.glob("*f2_to_inbalref_affine*"))[0])
anatomy_gad1b, _ = nrrd.read(list(master.glob("*f2_gad1b_to_inbalref_affine*"))[0])

# TODO avoid cringe
anatomy_h2b = np.swapaxes(anatomy_h2b, 0, 1)
anatomy_h2b = np.swapaxes(anatomy_h2b, 1, 2)
anatomy_h2b = anatomy_h2b[:, ::-1, :]
anatomy_h2b = anatomy_h2b[:, ::-1, ::-1]


anatomy_gad1b = np.swapaxes(anatomy_gad1b, 0, 1)
anatomy_gad1b = np.swapaxes(anatomy_gad1b, 1, 2)
anatomy_gad1b = anatomy_gad1b[:, ::-1, :]
anatomy_gad1b = anatomy_gad1b[:, ::-1, ::-1]

In [ ]:
atlas = BrainGlobeAtlas("mpin_zfish_1um")
atlas.annotation
# Annoying fixes required to deal with the Baier annotation mess. Will be cleaned in next version of the atlas.
atlas._annotation[[0, -1], :, :] = 0
atlas._annotation[:, [0, -1], :] = 0
atlas._annotation[:, :, [0, -1]] = 0
atlas._annotation[atlas._annotation == 0] = 15000
atlas.structures["root"]["id"] = 1

In [ ]:
p_starts = [460, 230, 270]
pw = 15
#  Specify axes limits over all dimensions:
bs = dict(
    frontal=(-10, atlas.shape[2] + 10),
    vertical=(-10, atlas.shape[1] + 10),
    sagittal=(-10, atlas.shape[0] + 10),
)

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["root", "hindbrain", "retina", "interpeduncular nucleus"],
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
    mask_slices=dict(frontal=slice(p_starts[0], p_starts[0] + pw), 
                     horizontal=slice(p_starts[1], p_starts[1] + pw) , 
                     sagittal=slice(p_starts[2], p_starts[2] + pw)),
    smooth_wnd=50,
)

In [ ]:
pw=10
f, axs = plotter.generate_projection_plots(title=True)
slices = [slice(p_starts[0], p_starts[0] + pw), 
          slice(p_starts[1], p_starts[1] + pw) , 
          slice(p_starts[2], p_starts[2] + pw),]
t = [False, False, True]
flip = [True, False, True]

ax_labs = "inf.  sup.", "pos.  ant.", "inf.  sup."
locs = [[0, 350], [0, 700], [0, 350]]

for i in range(3):
    sl = [slice(None, None),] * 3
    sl[i] = slices[i]
    
    sl_toplot = slices.copy()
    sl_toplot.pop(i)
    
    im = anatomy_h2b[tuple(sl)].mean(i)
    im_gad = anatomy_gad1b[tuple(sl)].mean(i) ** 0.3
    if t[i]:
        im = im.T
        im_gad = im_gad.T
    if flip[i]:
        im = im[::-1, :]
        im_gad = im_gad[::-1, :]
    axs[i].imshow(
        im,
        origin="lower",
        cmap="gray_r",
        vmin=0,
        vmax=100,
    )
    axs[i].imshow(
        im_gad,
        origin="lower",
        cmap=cmap_ring,
        vmin=0.5,
        vmax=2.5,
    )
    
    a = 0.2
    if t[i]:
        s1, s2 = [[sl_toplot[i].start, sl_toplot[i].stop] for i in [0, 1]]
    else:
        s1, s2 = [[sl_toplot[i].start, sl_toplot[i].stop] for i in [1, 0]]
    if flip[i]:
        s2 = [im_gad.shape[0] - s for s in s2]
    axs[i].axvspan(s1[0], s1[1], fc=COLS["sides"]["lf"], alpha=a, lw=0)
    axs[i].axhspan(s2[0], s2[1], fc=COLS["sides"]["lf"], alpha=a, lw=0)   
    
    xpos, ypos = locs[i]
    l = 150
    pltltr.add_scalebar(axs[i], xpos=xpos, ypos=ypos, xlen=l, ylen=-l, xlabel=f"{l} μm", ylabel=ax_labs[i], text_spacing_coef=0.15)
pltltr.savefig("gadline_views", folder="S1")